In [ ]:
import torch
from torch import nn
from torch.functional import F

In [ ]:
class LSTM(nn.Module):
	def __init__(self, embedding_callback, input_size, output_size, num_layers, num_classes):
		# Embeddings, which can be pretrained or normally trained
		self.embeddings = embedding_callback()
		# LSTM Layer
		self.lstm = nn.LSTM(input_size, output_size, num_layers)
		# Linear Layer
		self.linear = nn.Linear(output_size, num_classes)
	def forward(self, x):
		embedded_inputs = self.embeddings(x)
		lstm_output = self.lstm(embedded_inputs)
		linear_output = self.linear(lstm_output)
		return F.log_softmax(linear_output)